In [ ]:
%pip install transformers sentence-transformers numpy scikit-learn


# What Are Embeddings?
Definition: Embeddings are vector representations of text that capture semantic meaning.

Why Use Them?:

To find similarity between texts.

To enable tasks like search, clustering, and classification.

In [ ]:
from sentence_transformers import SentenceTransformer

# Load pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Example sentences
sentences = [
    "Artificial Intelligence is transforming industries.",
    "Machine learning is a subset of AI."
]

# Generate embeddings
embeddings = model.encode(sentences)

print("Embeddings:")
print(embeddings)
print("Shape of each embedding:", embeddings[0].shape)


In [ ]:
import chromadb
from chromadb.config import Settings

# Initialize ChromaDB client
client = chromadb.Client(Settings(
    chroma_db_impl="duckdb+parquet",  # Local storage
    persist_directory="db/chromadb_data"  # Directory to store data
))

# Create or get a collection
collection = client.get_or_create_collection(name="documents")

# Add embeddings to the collection
embeddings = [
    [0.1, 0.2, 0.3],
    [0.4, 0.5, 0.6],
    [0.7, 0.8, 0.9]
]
metadata = [{"id": "doc1"}, {"id": "doc2"}, {"id": "doc3"}]
documents = ["Document 1 text", "Document 2 text", "Document 3 text"]

collection.add(
    embeddings=embeddings,
    metadatas=metadata,
    documents=documents
)


In [ ]:
import pinecone

# Initialize Pinecone
pinecone.init(api_key="YOUR_API_KEY", environment="us-west1-gcp")

# Create a Pinecone index
index_name = "example-index"
if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension=3, metric="cosine")

index = pinecone.Index(index_name)

# Add embeddings to the index
embeddings = [
    {"id": "1", "values": [0.1, 0.2, 0.3]},
    {"id": "2", "values": [0.4, 0.5, 0.6]},
    {"id": "3", "values": [0.7, 0.8, 0.9]}
]
index.upsert(vectors=embeddings)


# Query for similar vectors
query_vector = [0.15, 0.25, 0.35]
results = index.query(vector=query_vector, top_k=2, include_metadata=True)
print(results)

